# Redes Neuronales Artificiales - Trabajo Práctico 2

### Guillermina Cabrol, Magalí Giansiracusa, Sofía Goldberg

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

seed = np.random.RandomState(seed=42)

## Introducción

En este trabajo buscamos obtener modelos de redes neuronales artificiales, basados en aprendizaje no supervisado, que logren aprender de un conjunto de documentos con descripciones de texto de compañías brasileñas. La idea es que lo modelos no solo logren aprender estos datos, sino también generalizar sobre nuevas instancias. 

Como cada entrada del conjunto de datos representa un documento, y la primer columna representa la categoría del mismo, 

In [36]:
import requests

r = requests.get('https://git.exactas.uba.ar/redes-neuronales/clases/-/raw/master/practicas/datos/tp2_training_dataset.csv')
data = np.loadtxt(r.iter_lines(), delimiter=',')

In [37]:
categorias = data[:,0]

X = data[:,1:]

X = (X - X.mean(axis=0)) / X.std()

## Reducción de dimensión

Nuestro primer objetivo será obtener dos modelos de redes neuronales artificiales que logren reducir la dimensión de los documentos en el conjunto de datos. Éstos van a diferir en la regla utilizada para actualizar los pesos: uno utilizará la **Regla de Oja** y otro la **Regla de Sanger**.

En este caso, dada una instancia de dimensión $N$, buscamos reducirla 

In [38]:
P = X.shape[0]
N = X.shape[1]
M = 9 # 9 dimensiones de salida

In [54]:
from training import PCA_train, oja_correction, sanger_correction

W = PCA_train(X, M, oja_correction, epochs=100, lr=0.001)

  0%|          | 0/100 [00:00<?, ?it/s]

 45%|████▌     | 45/100 [00:01<00:02, 22.62it/s]
